In [57]:
import pandas as pd
from pathlib import Path
import seaborn as sns
import hvplot.pandas
import requests
import json
import calendar
%matplotlib inline
from bokeh.models.formatters import DatetimeTickFormatter

In [2]:
sar_data_2020= Path("..\FInCen CSV Data\FinCen SAR Stats 2020 (Months).csv")
sar_2020_df= pd.read_csv(sar_data_2020, thousands=',', index_col='Year Month')
sar_2020_df

,State,Industry,Suspicious Activity,Count
Year Month,,,,
2020 January,Alabama,Casino/Card Club - Tribal Authorized Casino,Check,2
2020 January,Alabama,Casino/Card Club - Tribal Authorized Casino,Credit/Debit Card,2
2020 January,Alabama,Casino/Card Club - Tribal Authorized Casino,[Total],4
2020 January,Alabama,Depository Institution,ACH,96
2020 January,Alabama,Depository Institution,Advance Fee,4
...,...,...,...,...
2020 December,Wyoming,Money Services Business (MSB),[Total],28
2020 December,Wyoming,[Total],[Total],106
2020 December,[Total],[Total],[Total],123722


In [3]:
sar_2020_df=sar_2020_df.loc[sar_2020_df["Industry"]!="[Total]" ]
sar_2020_df=sar_2020_df.loc[sar_2020_df["Suspicious Activity"]!="[Total]" ]
sar_2020_df=sar_2020_df.loc[sar_2020_df["State"]!="[Total]" ]
sar_2020_df.loc[sar_2020_df.index.str.contains("[Total]") ]
sar_2020_df

,State,Industry,Suspicious Activity,Count
Year Month,,,,
2020 January,Alabama,Casino/Card Club - Tribal Authorized Casino,Check,2
2020 January,Alabama,Casino/Card Club - Tribal Authorized Casino,Credit/Debit Card,2
2020 January,Alabama,Depository Institution,ACH,96
2020 January,Alabama,Depository Institution,Advance Fee,4
2020 January,Alabama,Depository Institution,Business Loan,2
...,...,...,...,...
2020 December,Wyoming,Depository Institution,Other Fraud (Type),13
2020 December,Wyoming,Depository Institution,Wire,14
2020 December,Wyoming,Insurance Company,Wire,1


In [4]:
series_list1 = {
'LAUST010000000000003':'Alabama', 'LAUST020000000000003':'Alaska', 'LAUST040000000000003':'Arizona', 'LAUST050000000000003':'Arkansas', 'LAUST060000000000003':'California', 
'LAUST080000000000003':'Colorado', 'LAUST090000000000003':'Connecticut', 'LAUST100000000000003':'Delaware', 'LAUST120000000000003':'Florida', 'LAUST130000000000003':'Georgia', 
'LAUST150000000000003':'Hawaii', 'LAUST160000000000003':'Idaho', 'LAUST170000000000003':'Illinois', 'LAUST180000000000003':'Indiana', 
'LAUST190000000000003':'Iowa', 'LAUST200000000000003':'Kansas', 'LAUST210000000000003':'Kentucky', 'LAUST220000000000003':'Louisiana', 'LAUST230000000000003':'Maine', 
'LAUST240000000000003':'Maryland', 'LAUST250000000000003':'Massachusetts', 'LAUST260000000000003':'Michigan', 'LAUST270000000000003':'Minnesota', 'LAUST280000000000003':'Mississippi',
'LAUST290000000000003':'Missouri'} 
series_list2 = {
'LAUST300000000000003':'Montana', 'LAUST310000000000003':'Nebraska', 'LAUST320000000000003':'Nevada', 'LAUST330000000000003':'New Hampshire', 'LAUST340000000000003':'New Jersey', 
'LAUST350000000000003':'New Mexico', 'LAUST360000000000003':'New York', 'LAUST370000000000003':'North Carolina', 'LAUST380000000000003':'North Dakota', 'LAUST390000000000003':'Ohio', 'LAUST400000000000003':'Oklahoma', 
'LAUST410000000000003':'Oregon', 'LAUST420000000000003':'Pennsylvania', 'LAUST440000000000003':'Rhode Island', 'LAUST450000000000003':'South Carolina', 'LAUST460000000000003':'South Dakota', 'LAUST470000000000003':'Tennessee', 
'LAUST480000000000003':'Texas', 'LAUST490000000000003':'Utah', 'LAUST500000000000003':'Vermont', 'LAUST510000000000003':'Virginia', 
'LAUST530000000000003':'Washington', 'LAUST540000000000003':'West Virginia', 'LAUST550000000000003':'Wisconsin', 'LAUST560000000000003':'Wyoming',
}
df = pd.DataFrame(columns= ["State","Year","Month","Unemployment Rate"])

In [5]:
headers = {'Content-type': 'application/json'}
data1 = json.dumps({"seriesid": list(series_list1.keys()),"startyear":"2019", "endyear":"2021"})
p = requests.post('https://api.bls.gov/publicAPI/v1/timeseries/data/', data=data1, headers=headers)
json_data1 = json.loads(p.text)
data2 = json.dumps({"seriesid": list(series_list2.keys()),"startyear":"2019", "endyear":"2021"})
p = requests.post('https://api.bls.gov/publicAPI/v1/timeseries/data/', data=data2, headers=headers)
json_data2 = json.loads(p.text)

for series in json_data1['Results']['series']:
    state = series_list1[series['seriesID']]
    for item in series['data']:
        year = item['year']
        month = calendar.month_name[int(item['period'][1:3])]
        value = item['value']
        df = df.append({'State':state,'Year':year,'Month':month,'Unemployment Rate':value},ignore_index=True)

for series in json_data2['Results']['series']:
    state = series_list2[series['seriesID']]
    for item in series['data']:
        year = item['year']
        month = calendar.month_name[int(item['period'][1:3])]
        value = item['value']
        df = df.append({'State':state,'Year':year,'Month':month,'Unemployment Rate':value},ignore_index=True)
df

,State,Year,Month,Unemployment Rate
0,Alabama,2021,December,2.6
1,Alabama,2021,November,2.6
2,Alabama,2021,October,2.9
3,Alabama,2021,September,3.0
4,Alabama,2021,August,3.7
...,...,...,...,...
1795,Wyoming,2019,May,3.3
1796,Wyoming,2019,April,3.4
1797,Wyoming,2019,March,4.0
1798,Wyoming,2019,February,3.9


In [6]:
df_2020 = df.loc[df['Year']== '2020'].copy()
df_2020['Year Month'] = df_2020['Year'] +' '+ df_2020['Month']
df_2020 = df_2020[['Year Month', 'State','Unemployment Rate']]
df_2020.set_index('Year Month')

,State,Unemployment Rate
Year Month,,
2020 December,Alabama,4.1
2020 November,Alabama,4.4
2020 October,Alabama,4.9
2020 September,Alabama,6.0
2020 August,Alabama,6.7
...,...,...
2020 May,Wyoming,8.8
2020 April,Wyoming,5.3
2020 March,Wyoming,6.6


In [7]:
df_2020['Year Month']=pd.to_datetime(df_2020['Year Month'])
df_2020

,Year Month,State,Unemployment Rate
12,2020-12-01,Alabama,4.1
13,2020-11-01,Alabama,4.4
14,2020-10-01,Alabama,4.9
15,2020-09-01,Alabama,6.0
16,2020-08-01,Alabama,6.7
...,...,...,...
1783,2020-05-01,Wyoming,8.8
1784,2020-04-01,Wyoming,5.3
1785,2020-03-01,Wyoming,6.6
1786,2020-02-01,Wyoming,5.2


In [8]:
grouped_stats = sar_2020_df[["State", "Count"]].copy()
grouped_stats = grouped_stats.groupby('State').sum()
grouped_stats

,Count
State,
APO / DPO / FPO,38
Alabama,17941
Alaska,1574
American Samoa,11
Arizona,12116
...,...
Virginia,98438
Washington,16348
West Virginia,3251


In [34]:
mapped_data = grouped_stats.hvplot.bar(
    rot = 90,
    frame_width = 700, frame_height = 500,
    label = "FinCen Fraud SAR Reporting by State in 2020"
)

mapped_data

:Bars   [State]   (Count)

In [10]:
sar_2020_df= sar_2020_df.reset_index()
sar_2020_df

,Year Month,State,Industry,Suspicious Activity,Count
0,2020 January,Alabama,Casino/Card Club - Tribal Authorized Casino,Check,2
1,2020 January,Alabama,Casino/Card Club - Tribal Authorized Casino,Credit/Debit Card,2
2,2020 January,Alabama,Depository Institution,ACH,96
3,2020 January,Alabama,Depository Institution,Advance Fee,4
4,2020 January,Alabama,Depository Institution,Business Loan,2
...,...,...,...,...,...
16338,2020 December,Wyoming,Depository Institution,Other Fraud (Type),13
16339,2020 December,Wyoming,Depository Institution,Wire,14
16340,2020 December,Wyoming,Insurance Company,Wire,1
16341,2020 December,Wyoming,Money Services Business (MSB),Other Fraud (Type),17


In [11]:
sar_2020_df['Year Month']=pd.to_datetime(sar_2020_df['Year Month'])
sar_2020_df.dtypes

Year Month             datetime64[ns]
State                          object
Industry                       object
Suspicious Activity            object
Count                           int64
dtype: object

In [55]:
industry_by_ftype = sar_2020_df.groupby(['Industry', 'Suspicious Activity']).sum()
industry_by_ftype.hvplot.bar(stacked=True, frame_height=500, width=1050, rot=90, legend='top_left', yformatter='%.0f', label= 'Fraud Report by Industry Highlighted by the Type 2020')

:Bars   [Industry,Suspicious Activity]   (Count)

In [62]:
sar_2020_df_2= sar_2020_df.copy()
sar_2020_df_2['Year Month'] = sar_2020_df_2['Year Month'].astype(str)
sar_grouped_stats

Count
State           Year Month       
APO / DPO / FPO 2020-09-01      9
                2020-10-01     11
                2020-11-01      6
                2020-12-01     12
Alabama         2020-01-01   1004
...                           ...
Wyoming         2020-08-01    118
                2020-09-01     89
                2020-10-01     81
                2020-11-01     72
                2020-12-01    106

[696 rows x 1 columns]

In [63]:
sar_grouped_stats= sar_2020_df_2[['State', 'Count', 'Year Month']].copy()
sar_grouped_stats= sar_grouped_stats.groupby(['State','Year Month',]).sum()
sar_grouped_stats= sar_grouped_stats.sort_index(level=1)
sar_grouped_stats.hvplot.bar(stacked=True, frame_height=500, width=1050, rot=90, legend='left', yformatter='%.0f', label= 'Fraud Report by Industry Highlighted by the Type 2020')

:Bars   [State,Year Month]   (Count)

In [23]:
grouped_df= df_2020.groupby(['State','Year Month']).sum()
grouped_df

Unemployment Rate
State   Year Month                  
Alabama 2020-01-01               3.7
        2020-02-01               3.4
        2020-03-01               4.0
        2020-04-01              13.0
        2020-05-01              10.2
...                              ...
Wyoming 2020-08-01               5.2
        2020-09-01               5.1
        2020-10-01               4.3
        2020-11-01               4.7
        2020-12-01               5.0

[600 rows x 1 columns]

In [31]:
sar_stats_2020_clean= sar_2020_df.drop(columns= 'Suspicious Activity')
sar_stats_2020_clean

,Year Month,State,Industry,Count
0,2020-01-01,Alabama,Casino/Card Club - Tribal Authorized Casino,2
1,2020-01-01,Alabama,Casino/Card Club - Tribal Authorized Casino,2
2,2020-01-01,Alabama,Depository Institution,96
3,2020-01-01,Alabama,Depository Institution,4
4,2020-01-01,Alabama,Depository Institution,2
...,...,...,...,...
16338,2020-12-01,Wyoming,Depository Institution,13
16339,2020-12-01,Wyoming,Depository Institution,14
16340,2020-12-01,Wyoming,Insurance Company,1
16341,2020-12-01,Wyoming,Money Services Business (MSB),17


In [36]:
sar_stats_2020_clean = sar_stats_2020_clean.reset_index()
sar_stats_2020_clean['Year Month'] = pd.to_datetime(sar_stats_2020_clean['Year Month'], infer_datetime_format = True)
sar_stats_2020_clean['Count'] = sar_stats_2020_clean['Count'].astype(float)

sar_stats_2020_clean.dtypes

level_0                int64
index                  int64
Year Month    datetime64[ns]
State                 object
Industry              object
Count                float64
dtype: object

In [37]:
grouped_stats_state = sar_stats_2020_clean[["State", "Count", "Year Month"]]

grouped_stats_state = grouped_stats_state.groupby(['Year Month', 'State']).sum()
grouped_stats_state

Count
Year Month State                 
2020-01-01 Alabama         1004.0
           Alaska           132.0
           American Samoa     2.0
           Arizona          884.0
           Arkansas         329.0
...                           ...
2020-12-01 Virginia        9475.0
           Washington      1575.0
           West Virginia    294.0
           Wisconsin        787.0
           Wyoming          106.0

[696 rows x 1 columns]

In [38]:
grouped_stats_ym = sar_stats_2020_clean.groupby(['Year Month']).sum()
grouped_stats_ym

,level_0,index,Count
Year Month,,,
2020-01-01,928203,928203,111567.0
2020-02-01,2775078,2775078,103631.0
2020-03-01,4609722,4609722,116253.0
2020-04-01,6426732,6426732,111134.0
2020-05-01,7701915,7701915,91301.0
2020-06-01,9939478,9939478,107781.0
2020-07-01,12284298,12284298,127104.0
2020-08-01,14353254,14353254,126465.0
2020-09-01,16153511,16153511,127634.0


In [39]:
grouped_stats_ym = grouped_stats_ym.rename(columns = {'Count' : 'Total'})
grouped_stats_ym

,level_0,index,Total
Year Month,,,
2020-01-01,928203,928203,111567.0
2020-02-01,2775078,2775078,103631.0
2020-03-01,4609722,4609722,116253.0
2020-04-01,6426732,6426732,111134.0
2020-05-01,7701915,7701915,91301.0
2020-06-01,9939478,9939478,107781.0
2020-07-01,12284298,12284298,127104.0
2020-08-01,14353254,14353254,126465.0
2020-09-01,16153511,16153511,127634.0


In [40]:
pd.set_option("display.max_rows", 10, "display.max_columns", None)
grouped_stats_state = grouped_stats_state.rename(columns = {'Count' : 'Reports'})
grouped_stats_state

Reports
Year Month State                  
2020-01-01 Alabama          1004.0
           Alaska            132.0
           American Samoa      2.0
           Arizona           884.0
           Arkansas          329.0
...                            ...
2020-12-01 Virginia         9475.0
           Washington       1575.0
           West Virginia     294.0
           Wisconsin         787.0
           Wyoming           106.0

[696 rows x 1 columns]

In [41]:
pd.set_option("display.max_rows", 10, "display.max_columns", None)
fraud_data = pd.merge(grouped_stats_state, grouped_stats_ym, left_on = 'Year Month',right_index=True)
fraud_data

Reports   level_0     index     Total
Year Month State                                                
2020-01-01 Alabama          1004.0    928203    928203  111567.0
           Alaska            132.0    928203    928203  111567.0
           American Samoa      2.0    928203    928203  111567.0
           Arizona           884.0    928203    928203  111567.0
           Arkansas          329.0    928203    928203  111567.0
...                            ...       ...       ...       ...
2020-12-01 Virginia         9475.0  21166117  21166117  123722.0
           Washington       1575.0  21166117  21166117  123722.0
           West Virginia     294.0  21166117  21166117  123722.0
           Wisconsin         787.0  21166117  21166117  123722.0
           Wyoming           106.0  21166117  21166117  123722.0

[696 rows x 4 columns]

In [42]:
fraud_data['Reporting Rate'] = round((fraud_data['Reports']/fraud_data['Total'])*100,3)
fraud_data

Reports   level_0     index     Total  \
Year Month State                                                   
2020-01-01 Alabama          1004.0    928203    928203  111567.0   
           Alaska            132.0    928203    928203  111567.0   
           American Samoa      2.0    928203    928203  111567.0   
           Arizona           884.0    928203    928203  111567.0   
           Arkansas          329.0    928203    928203  111567.0   
...                            ...       ...       ...       ...   
2020-12-01 Virginia         9475.0  21166117  21166117  123722.0   
           Washington       1575.0  21166117  21166117  123722.0   
           West Virginia     294.0  21166117  21166117  123722.0   
           Wisconsin         787.0  21166117  21166117  123722.0   
           Wyoming           106.0  21166117  21166117  123722.0   

                           Reporting Rate  
Year Month State                           
2020-01-01 Alabama                  0.900  
           Alaska                   0.118  
           American Samoa           0.002  
           Arizona                  0.792  
           Arkansas                 0.295  
...                                   ...  
2020-12-01 Virginia                 7.658  
           Washington               1.273  
           West Virginia            0.238  
           Wisconsin                0.636  
           Wyoming                  0.086  

[696 rows x 5 columns]

In [70]:
fraud_data_rate = fraud_data[['Reporting Rate']].copy()
formatter = DatetimeTickFormatter(months='%b %Y')
fraud_plot= fraud_data_rate.hvplot(groupby='State' ,yformatter = '%.3f', xformatter=formatter, label= 'Fraud Report by State 2020')
fraud_plot

:DynamicMap   [State]
   :Curve   [Year Month]   (Reporting Rate)

In [71]:
df_2020.sort_index(inplace=True,ascending=False)
df_2020 = df_2020.astype({'Unemployment Rate':'float'})
formatter = DatetimeTickFormatter(months='%b %Y')
unemployment_plot= df_2020.hvplot(y='Unemployment Rate', x = 'Year Month',groupby='State', rot = 90, label= 'Unemployment by Industry Highlighted by the Type 2020')
unemployment_plot

:DynamicMap   [State]
   :Curve   [Year Month]   (Unemployment Rate)

In [77]:
fraud_plot * unemployment_plot


:DynamicMap   [State]
   :Overlay
      .Curve.Fraud_Report_by_State_2020                            :Curve   [Year Month]   (Reporting Rate)
      .Curve.Unemployment_by_Industry_Highlighted_by_the_Type_2020 :Curve   [Year Month]   (Unemployment Rate)